# An LLM-Approach to Semantic Clustering and Topic Modeling of Academic Literature

[Clustering](https://en.wikipedia.org/wiki/Cluster_analysis) stands as a fundamental task in unsupervised learning, where the goal is to group unlabeled data into related categories; whereas [Topic Modeling](https://en.wikipedia.org/wiki/Topic_model) focuses on identifying thematic structures within a collection of documents. These techniques find applications across various domains, enabling tasks such as information retrieval, anomaly detection, trend analysis, and biomedical research.

This notebook provides an end-to-end guide to building an LLM-based pipeline for automatic categorization of research articles into latent topics using open-source tools and models. Our playground is a  [dataset of 25,000 research arXiv publications](https://huggingface.co/datasets/dcarpintero/arxiv.cs.CL.25k) from Computational Linguistics (Natural Language Processing) published before July 2024.

At its core, the clustering problem relies on finding similar examples. This is a natural task for embeddings, as they capture the semantic relationships in a corpus, and can be provided as input features to a clustering algorithm to establish similarity links among the examples. We begin by transforming the `title:abstract` pairs of our dataset into an embeddings representation using  [Jina-Embeddings-v2](https://arxiv.org/abs/2310.19923), a BERT-ALiBi based attention model supporting 8192 sequence length, and subsequently applying HDBSCAN [2] in a reduced dimensional space. Topic modeling is then performed at cluster level using a random subset of `titles` within each cluster. This latter process combines [LangChain](https://www.langchain.com/) and [Pydantic](https://docs.pydantic.dev/) with [Mistral-7B-Instruct](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) to construct a topic pipeline that generates structured `JSON` output.

To measure the clustering and topic modeling effectiveness, we visualize the outcomes after further applying [UMAP](https://en.wikipedia.org/wiki/Uniform_Manifold_Approximation_and_Projection) [1] dimensionality reduction.

The results hint at emerging research topics in the field of Computational Linguistics such as: '**Vision-Language-Models**', '**Medical NLP**', '**Large Language Model Alignment via Human Preference**', '**Multilingual Language Models**', '**Attacks on LLMs**', '**Bias in LLMs**', '**LLM-based Agents**', '**Hallucination in LLMs**', and '**Language Model Compression and Acceleration**'. A similar approach could be used to create a more granular arXiv taxonomy for categorization and retrieval of sub-topics in the field of Computational Linguistics.

<figure>
  <img style="margin: 0 auto; display: block;" src="https://cdn-uploads.huggingface.co/production/uploads/64a13b68b14ab77f9e3eb061/iE3e4VJSY84JyyTR9krmf.png">
  <figcaption style="text-align: center;">LLM-based Pipeline for Semantic Clustering and Topic Modeling of Academic Literature </figcaption>
</figure>

In [1]:
%%capture
%pip install altair datasets hdbscan scikit-learn umap-learn sentence-transformers --quiet

## 1. Embeddings Transformation

Embeddings are numerical representations of real-world objects like text, images, and audio that encapsulate semantic information of the data they represent. They are used by AI models to understand complex knowledge domains in downstream applications such as clustering, as well as information retrieval and classification tasks.

We implement this step with [Jina-Embeddings-v2](https://arxiv.org/abs/2310.19923), an open-source text embedding model capable of accommodating up to 8192 tokens. This provides a sufficiently large sequence length for `title:abstract` pairs and other document sections that might be relevant.

To overcome the conventional 512-token limit, Jina-Embeddings-v2 incorporates bidirectional [ALiBi](https://arxiv.org/abs/2108.12409) into the BERT framework. AliBi (Attention with Linear Biases) enables input length extrapolation (i.e. sequences exceeding 2048 tokens) by encoding positional information directly within the
self-attention layer, instead of introducing positional embeddings. In practice, it biases query-key attention scores with a penalty that is proportional to their distance, ensuring that proximate tokens demonstrate stronger mutual attention.

The semantic similarity between corpora can be trivially computed as the inner product of the embeddings. In the following heat map each entry [x, y] is colored based on said embeddings product for sentences [x] and [y].

<figure>
  <img style="margin: 0 auto; display: block;" src="https://cdn-uploads.huggingface.co/production/uploads/64a13b68b14ab77f9e3eb061/4djmELIe2LkZ8_Tofc91Q.png">
  <figcaption style="text-align: center;">Semantic Similary in arXiv 'titles' w/ Jina-Embeddings-v2</figcaption>
</figure>

In [79]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


configuration_bert.py:   0%|          | 0.00/8.24k [00:00<?, ?B/s]

modeling_bert.py:   0%|          | 0.00/97.7k [00:00<?, ?B/s]

In [80]:
import pandas as pd

ARXIV_JSON='./arxiv.cs.CL.25k.desc.json'
ARXIV_EMBEDDINGS_JSONL='./arxiv.cs.CL.embeddings.25k.jina.jsonl'
BATCH_SIZE=64

df = pd.read_json(ARXIV_JSON)
embeddings = model.encode(df['title'] + ': ' + df['abstract'],
                          batch_size=BATCH_SIZE,
                          show_progress_bar=True)
embeddings.shape

Batches:   0%|          | 0/393 [00:00<?, ?it/s]

(25107, 768)

In [81]:
from datasets import Dataset

df['embeddings'] = embeddings.tolist()
df.to_json(ARXIV_EMBEDDINGS_JSONL, orient='records', lines=True)

In [83]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'doc_url', 'title', 'publication_date', 'update_date', 'authors', 'category_primary', 'category_all', 'abstract', 'embeddings'],
    num_rows: 25107
})

As an alternative you might just load the prepared dataset with embeddings from HuggingFace.

In [37]:
from datasets import load_dataset
import tqdm as notebook_tqdm

ds = load_dataset("dcarpintero/arxiv.cs.CL.25k.embeddings.jina", split="train")

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [38]:
ds

Dataset({
    features: ['id', 'doc_url', 'title', 'publication_date', 'update_date', 'authors', 'category_primary', 'category_all', 'abstract', 'embeddings'],
    num_rows: 10000
})

## 2. Projecting Embeddings for Dimensionality Reduction

We then project our (`title:abstract`) embeddings pairs from a high-dimensional space (768) to a lower-dimensional one (5) using
[dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction). This process will reduce the computational complexity and memory usage during clustering.

To implement this step, we use [UMAP](https://en.wikipedia.org/wiki/Nonlinear_dimensionality_reduction#Uniform_manifold_approximation_and_projection) [1], a popular technique known for its effectiveness in preserving both the local and global data structures. In practice, this makes it a preferred choice for handling complex datasets with high-dimensional embeddings.

In [84]:
import umap

umap_reducer = umap.UMAP(n_neighbors=100,
                         n_components=5,
                         min_dist=0.1,
                         metric='cosine')

umap_embedding = umap_reducer.fit_transform(ds['embeddings'])

In our implementation, we configure UMAP with:
- `n_neighbors=100` to consider 100 nearest neighbors for each point (arXiv publication);
- `n_components=5` to reduce the embeddings from 768 to 5 dimensions;
- `min_dist=0.1` to maintain a balance between the local and global structure; and,
- `metric='cosine'` to measure the distance between points using the cosine similarity metric.

Note that when we apply HDBSCAN clustering in the next step, the clusters found will be influenced by how UMAP preserved the local structures. A smaller `n_neighbors` value means UMAP will focus more on local structures, whereas a larger value allows to capture more global representations, which might be beneficial for understanding overall patterns in the data.

## 3. Semantic Clustering

This section shows how to use the reduced (`title:abstract`) embeddings pairs as input features of a clustering algorithm. This allows for the identification of related categories based on the distance between the provided embeddings.

We have opted for [HDBSCAN](https://en.wikipedia.org/wiki/HDBSCAN) (Hierarchical Density-Based Spatial Clustering of Applications with Noise) [2], an advanced clustering algorithm that extends DBSCAN by adapting to varying density clusters. Unlike K-Means which requires pre-specifying the number of clusters, HDBSCAN has only one important hyperparameter, `n`, which establishes the minimum number of examples to include in a cluster. As a density-based method, it can also detect outliers in the data.

HDBSCAN works by first transforming the data space according to the density of the data points, making denser regions (areas where data points are close together in high numbers) more attractive for cluster formation. The algorithm then builds a hierarchy of clusters based on the minimum cluster size established by the hyperparameter `n`. This allows it to distinguish between noise (sparse areas) and dense regions (potential clusters). Finally, HDBSCAN condenses this hierarchy to derive the most persistent clusters, efficiently identifying clusters of different densities and shapes.

Note that while we define a minimum cluster size similar to the number of neighbors in UMAP, in practice they do not need to be equal.

In [85]:
import hdbscan

hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=100,
                                metric='euclidean',
                                cluster_selection_method='eom')
clusters = hdbscan_model.fit_predict(umap_embedding)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


We prepare the dataset for visualization by further reducing the number of dimensions, in this case from '5' to '2'.

In [86]:
import pandas as pd

reduced_embeddings = umap.UMAP(n_neighbors=100, n_components=2, min_dist=0.1, metric='cosine').fit_transform(ds['embeddings'])
df = pd.DataFrame(reduced_embeddings, columns=['x', 'y'])
df['cluster'] = clusters
df['title'] = ds['title']
df['publication_date'] = ds['publication_date']

df = df[df['cluster'] != -1] # remove outliers

In [95]:
df.head(10)

,x,y,cluster,title,topic
0,6.794572,0.783290,22,Planetarium: A Rigorous Benchmark for Translat...,LLM-based Agents
1,9.353059,5.155807,5,InternLM-XComposer-2.5: A Versatile Large Visi...,Multimodal Vision-Language Processing
2,4.567746,1.926857,6,BACON: Supercharge Your VLM with Bag-of-Concep...,Hallucination in Large Language Models
4,4.667935,2.019940,6,LLM Internal States Reveal Hallucination Risk ...,Hallucination in Large Language Models
5,10.056472,2.634769,11,Evaluating Automatic Metrics with Incremental ...,Multilingual Language Models and Machine Trans...
6,4.547708,3.714908,18,How Similar Are Elected Politicians and Their ...,Fake News Detection
9,3.869952,0.892294,1,Self-Evaluation as a Defense Against Adversari...,Attacks on Large Language Models
10,3.948679,0.714475,1,Single Character Perturbations Break LLM Align...,Attacks on Large Language Models
12,10.056930,0.311966,30,How Does Quantization Affect Multilingual LLMs?,Language Model Compression and Acceleration
14,7.311922,0.215009,25,Fine-Tuning with Divergent Chains of Thought B...,Mathematical Reasoning in Language Models


In [88]:
# number of clusters
len(df['cluster'].unique())


32

## 4. Topic Modeling with LLMs

Having performed the clustering step, we now illustrate how to identify the topic of each cluster by combining an LLM such as [Mistral-7B-Instruct](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3) with [Pydantic](https://docs.pydantic.dev/) and [LangChain](https://www.langchain.com/) to create a topic modeling pipeline.

In [11]:
%pip install huggingface_hub langchain langchain_huggingface --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.8 MB/s eta 0:00:00


### 4.1 Pydantic Model

[Pydantic Models](https://docs.pydantic.dev/latest/concepts/models/) are classes that derive from `pydantic.BaseModel`, defining fields as type-annotated attributes. They bear a strong resemblance to `Python` dataclasses. However, they have been designed with subtle but significant differences that optimize various operations such as validation, serialization, and `JSON` schema generation. Our `Topic` class defines a field named `category`. This will generate output in a structured format, rather than a free-form text block, facilitating easier processing and analysis of the topic modeling results.

In [12]:
from pydantic import BaseModel, Field

class Topic(BaseModel):
    """
    Pydantic Model to generate an structured Topic Model
    """
    label: str = Field(..., description="Identified topic")

### 4.2 LangChain Prompt Template

[LangChain Prompt Templates](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates) are pre-defined recipes for generating prompts for language models.

In [13]:
from langchain_core.prompts import PromptTemplate

topic_prompt = """
    You are a helpful Research Engineer. Your task is to analyze a set of research paper titles related to Natural Language Processing and
    determine the overarching topic of the cluster. Based on the titles provided, you should identify and label the most relevant topic.
    The response should be concise, clearly stating the single  identified topic. Format your response in JSON as indicated in the 'EXPECTED OUTPUT' section below.
    No additional information or follow-up questions are needed.

    EXPECTED OUTPUT:
    {{"label": "Topic Name"}}

    TITLES:
    {titles}
    """

### 4.3 Inference of Topic Identification

This section illustrates how to compose a topic pipeline using the [LangChain Expression Language (LCEL)](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel).

In [14]:
import os
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [15]:
import os

from langchain.chains import LLMChain
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.output_parsers import PydanticOutputParser

from typing import List

def TopicModeling(titles: List[str]) -> str:
    """
    Infer the common topic of the given titles w/ LangChain, Pydantic, Mistral-7B-Instruct
    """
    repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
    llm = HuggingFaceEndpoint(
        repo_id=repo_id,
        temperature=0.2,
        huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
    )
    prompt = PromptTemplate.from_template(topic_prompt)
    parser = PydanticOutputParser(pydantic_object=Topic)

    topic_chain = prompt | llm | parser
    return topic_chain.invoke({"titles": titles})

To enable the model to infer the topic of each cluster, we provide a random subset of 25 paper titles from each cluster as input.

In [89]:
%%capture
topics = []
for i, cluster in df.groupby('cluster'):
    titles = cluster['title'].sample(25).tolist()
    topic = TopicModeling(titles)
    topics.append(topic.label)

Lets assign each arXiv publication to each cluster, and see what are the top 15 topics.

In [90]:
n_clusters = len(df['cluster'].unique())

topic_map = dict(zip(range(n_clusters), topics))
df['topic'] = df['cluster'].map(topic_map)

In [97]:
df['topic'].value_counts()

topic
Multimodal Vision-Language Processing                   1313
Speech Recognition and Synthesis                        1200
Medical NLP                                             1070
Multilingual Language Models and Machine Translation     860
Attacks on Large Language Models                         781
Knowledge Graphs                                         581
Mathematical Reasoning in Language Models                565
Bias in Language Models                                  429
Named Entity Recognition                                 371
Task-Oriented Dialogue Systems                           325
Fake News Detection                                      313
Large Language Model Alignment via Human Preference      310
Emotion Recognition and Analysis                         308
Transformers                                             275
Hate Speech Detection                                    257
Aspect-Based Sentiment Analysis                          235
LLM-based Agents  

## 5. Visualization

In [19]:
%pip install vegafusion[embed]>=1.5.0 --quiet

import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [93]:
chart = alt.Chart(df).mark_circle(size=5).encode(
    x='x',
    y='y',
    color='topic:N',
    tooltip=['title', 'topic']
).interactive().properties(
    title='25K arXiv Publications | Clustering and Topic Modeling | Jina-Embeddings | Mistral-7B',
    width=600,
    height=400,
)
chart.display()

alt.Chart(...)

----